# Creación de Sesión

1° PASO: Importamos módulos de apache spark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

2° PASO: Creamos las session de apache spark en una variable

In [2]:
spark = SparkSession.builder\
    .master("local[*]") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

3° PASO: Verificamos la versión de apache spark

In [3]:
spark

# Poblando Capa Staging

## Poblando 'Products'

4° PASO: Crear un dataframe

1. Crear la estructura del dataframe
2. Declarar en una variable la ruta del archivo
3. Leer el archivo de origen
4. Mostrar la estructura del dataframe
5. Mostrar los datos del dataframe
6. Cantidad de registros del dataframe
7. Mostrar las estadísticas básicas de un campo determinado

In [4]:
# 4.1 Estructura del dataframe.
df_schema_products = StructType([
    StructField("product_ID", IntegerType(),True),
    StructField("product_type", StringType(),True),
    StructField("product_name", StringType(),True),
    StructField("size", StringType(),True),
    StructField("colour", StringType(),True),
    StructField("price", DoubleType(),True),
    StructField("quantity", IntegerType(),True),
    StructField("description", StringType(),True)
])

In [5]:
# 4.2 Definimos ruta del archivo
#Archivo en Cloud Storage - Google Cloud Platform
ruta_products = "gs://curso-bigdata/datalake/raw/products.csv"

In [7]:
# 4.3 Leer el archivo de origen
products = spark.read.format("CSV")\
    .option("header","true")\
    .option("delimiter",",")\
    .schema(df_schema_products)\
    .load(ruta_products)

In [8]:
# 4.4 Mostramos la estructura del dataframe.
products.printSchema()

root
 |-- product_ID: integer (nullable = true)
 |-- product_type: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- size: string (nullable = true)
 |-- colour: string (nullable = true)
 |-- price: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- description: string (nullable = true)



In [9]:
# 4.5 Mostraremos todas las columnas de las 5 primeras filas del dataframe.
products.show(5,truncate=0)

+----------+------------+------------+----+------+-----+--------+--------------------------------------------+
|product_ID|product_type|product_name|size|colour|price|quantity|description                                 |
+----------+------------+------------+----+------+-----+--------+--------------------------------------------+
|0         |Shirt       |Oxford Cloth|XS  |red   |114.0|66      |A red coloured, XS sized, Oxford Cloth Shirt|
|1         |Shirt       |Oxford Cloth|S   |red   |114.0|53      |A red coloured, S sized, Oxford Cloth Shirt |
|2         |Shirt       |Oxford Cloth|M   |red   |114.0|54      |A red coloured, M sized, Oxford Cloth Shirt |
|3         |Shirt       |Oxford Cloth|L   |red   |114.0|69      |A red coloured, L sized, Oxford Cloth Shirt |
|4         |Shirt       |Oxford Cloth|XL  |red   |114.0|47      |A red coloured, XL sized, Oxford Cloth Shirt|
+----------+------------+------------+----+------+-----+--------+--------------------------------------------+
o

In [10]:
# 4.6 Mostraremos todos los datos del dataframe.
num_rows = products.count()
print("La cantidad de registro del dataframe es: ", num_rows)

La cantidad de registro del dataframe es:  1260


In [11]:
# 4.7 Estadísticas de un campo determinado.
products.describe('price').show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|              1260|
|   mean|105.80555555555556|
| stddev| 9.704422855767938|
|    min|              90.0|
|    max|             119.0|
+-------+------------------+



5° PASO: Guardar el dataframe en un ruta de la capa staging

In [12]:
# Guardar el dataframe en un ruta de la capa staging
#Archivo en Cloud Storage - Google Cloud Platform
ruta_products_staging = "gs://curso-bigdata/datalake/staging/products/"

products.write.mode("overwrite")\
    .format("parquet")\
    .partitionBy("product_type","size")\
    .save(ruta_products_staging)

## Poblanco 'Customers'

In [13]:
# Estructura del dataframe.
df_schema_customers = StructType([
    StructField("customer_id",IntegerType(),True),
    StructField("customer_name",StringType(),True),
    StructField("gender",StringType(),True),
    StructField("age",IntegerType(),True),
    StructField("home_address",StringType(),True),
    StructField("zip_code",IntegerType(),True),
    StructField("city",StringType(),True),
    StructField("state",StringType(),True),
    StructField("country",StringType(),True)
])

In [14]:
# Definimos ruta del archivo
ruta_customers = "gs://curso-bigdata/datalake/raw/customers.csv"

In [15]:
# Leer el archivo de origen
customers = spark.read.format("CSV")\
    .option("header","true")\
    .option("delimiter",",")\
    .schema(df_schema_customers)\
    .load(ruta_customers)

In [16]:
# Mostramos la estructura del dataframe.
customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- home_address: string (nullable = true)
 |-- zip_code: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)



In [17]:
# Mostraremos todas las columnas de las 5 primeras filas del dataframe.
customers.show(5,truncate=0)

+-----------+-------------------+-----------+---+------------------------------+--------+-----------------+----------------------------+---------+
|customer_id|customer_name      |gender     |age|home_address                  |zip_code|city             |state                       |country  |
+-----------+-------------------+-----------+---+------------------------------+--------+-----------------+----------------------------+---------+
|1          |Leanna Busson      |Female     |30 |8606 Victoria TerraceSuite 560|5464    |Johnstonhaven    |Northern Territory          |Australia|
|2          |Zabrina Harrowsmith|Genderfluid|69 |8327 Kirlin SummitApt. 461    |8223    |New Zacharyfort  |South Australia             |Australia|
|3          |Shina Dullaghan    |Polygender |59 |269 Gemma SummitSuite 109     |5661    |Aliburgh         |Australian Capital Territory|Australia|
|4          |Hewet McVitie      |Bigender   |67 |743 Bailey GroveSuite 141     |1729    |South Justinhaven|Queensland 

In [18]:
# Mostraremos todos los datos del dataframe.
num_rows = customers.count()
print("La cantidad de registro del dataframe es: ", num_rows)

La cantidad de registro del dataframe es:  1000


In [19]:
# Estadísticas de un campo determinado.
customers.describe('age').show()

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|              1000|
|   mean|             49.86|
| stddev|17.647828360618387|
|    min|                20|
|    max|                80|
+-------+------------------+



In [22]:
# Guardar el dataframe en un ruta de la capa staging
ruta_customers_staging = "gs://curso-bigdata/datalake/staging/customers/"

customers.write.mode("overwrite")\
    .format("parquet")\
    .partitionBy("country","state","city")\
    .save(ruta_customers_staging)

## Poblanco 'Orders'

In [23]:
# Estructura del dataframe.
df_schema_orders = StructType([
    StructField("order_id",IntegerType(),True),
    StructField("customer_id",IntegerType(),True),
    StructField("payment",DoubleType(),True),
    StructField("order_date",DateType(),True),
    StructField("delivery_date",DateType(),True)
])

In [24]:
# Definimos ruta del archivo
ruta_orders = "gs://curso-bigdata/datalake/raw/orders.csv"

In [25]:
# Leer el archivo de origen
orders = spark.read.format("CSV")\
    .option("header","true")\
    .option("delimiter",",")\
    .schema(df_schema_orders)\
    .load(ruta_orders)

In [26]:
# Mostramos la estructura del dataframe.
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- payment: double (nullable = true)
 |-- order_date: date (nullable = true)
 |-- delivery_date: date (nullable = true)



In [27]:
# Mostraremos todas las columnas de las 5 primeras filas del dataframe.
orders.show(5,truncate=0)

+--------+-----------+-------+----------+-------------+
|order_id|customer_id|payment|order_date|delivery_date|
+--------+-----------+-------+----------+-------------+
|1       |64         |30811.0|2021-08-30|2021-09-24   |
|2       |473        |50490.0|2021-02-03|2021-02-13   |
|3       |774        |46763.0|2021-10-08|2021-11-03   |
|4       |433        |39782.0|2021-05-06|2021-05-19   |
|5       |441        |14719.0|2021-03-23|2021-03-24   |
+--------+-----------+-------+----------+-------------+
only showing top 5 rows



In [28]:
# Mostraremos todos los datos del dataframe.
num_rows = orders.count()
print("La cantidad de registro del dataframe es: ", num_rows)

La cantidad de registro del dataframe es:  1000


In [29]:
# Estadísticas de un campo determinado.
orders.describe('payment').show()

+-------+------------------+
|summary|           payment|
+-------+------------------+
|  count|              1000|
|   mean|         33972.936|
| stddev|14451.609046882906|
|    min|           10043.0|
|    max|           59910.0|
+-------+------------------+



In [31]:
# Guardar el dataframe en un ruta de la capa staging
ruta_orders_staging = "gs://curso-bigdata/datalake/staging/orders/"

# Del campo order-date se extrae el año y mes para usarlo como partición al momento de escribir 
orders = orders.withColumn('year',year(col('order_date')))
orders = orders.withColumn('month',month(col('order_date')))

orders.write.mode("overwrite")\
    .format("parquet")\
    .partitionBy("year","month")\
    .save(ruta_orders_staging)